In [1]:
using Pkg
Pkg.activate(".")
using ITensors
using ITensorTDVP
using Observers
using DelimitedFiles

  Activating environment at `~/Desktop/TamaITensor/TDVP/TDVP1_Fishman/Project.toml`
┌ Info: Precompiling ITensorTDVP [25707e16-a4db-4a07-99d9-4d67b7af0342]
└ @ Base loading.jl:1342


# Load system and chain parameters

In [2]:
#Chain parameters from file
coups = readdlm("../Chain_mappings/WSCP_MC_T0_coups.dat");
freqs = readdlm("../Chain_mappings/WSCP_MC_T0_freqs.dat");
#system energy gap
eps = 138.;

In [3]:
sys = siteinds("S=1/2",1);
NChain = 86;
env = siteinds("Boson", dim=6, NChain);
NN = NChain + 1;
sysenv = vcat(sys,env);

In [4]:
stateSys = ["Up"];
stateEnv = ["0" for n=1:NChain];
stateSE = vcat(stateSys,stateEnv);
psi0 = productMPS(sysenv,stateSE);

In [5]:
wantedDim = 8
psiExt = copy(psi0);
for n in 1:NN-1
   a = commonind(psiExt[n],psiExt[n+1])
   tagsa = tags(a)
   add_indx = Index(wantedDim, tags= tagsa)
   psiExt[n]=psiExt[n]*delta(a,add_indx)
   psiExt[n+1]=psiExt[n+1]*delta(a,add_indx)
end
dot(psi0,psiExt)

1.0

In [6]:
psi0=copy(psiExt);

In [7]:
#asymptotic frequency
MC_N = 6;
NP_Chain = NN - MC_N;
OO = 500.;
alphas_MC = readdlm("../MC_Pars/alphas_6.dat")
betas_MC = readdlm("../MC_Pars/betas_6.dat")
coups_MC = readdlm("../MC_Pars/coupls_6.dat")
gammas = OO * alphas_MC[:,1];
eff_freq = [OO + 1im * g for g in gammas];
eff_gs = OO * betas_MC[:,2];
eff_coup = OO/2* (coups_MC[:,1]+ 1im *coups_MC[:,2]);

#Definition of the TEDOPA+MC Hamiltonian (MPO form)
#Lavoriamo qui
ampo = OpSum()
#system Hamiltonian
ampo += eps,"Sz",1;
#system-env interaction
#!Sx = 0.5 σx
ampo += 2*coups[1],"Sx",1,"Adag",2
ampo += 2*coups[1],"Sx",1,"A",2

#Primary chain local Hamiltonians
for j=2:NP_Chain
   ampo += freqs[j-1],"N",j
end

for j=2:NP_Chain-1
   ampo += coups[j],"A",j,"Adag",j+1
   ampo += coups[j],"Adag",j,"A",j+1
end
#################################

#Markovian closure Hamiltonian
for j=1:MC_N
   ampo += eff_freq[j],"N",NP_Chain+j
end

for j=1:MC_N-1
   ampo += eff_gs[j],"A",NP_Chain+j,"Adag",NP_Chain+j+1
   ampo += eff_gs[j],"Adag",NP_Chain+j,"A",NP_Chain+j+1
end

#################################

#Primary chain - MC interaction
for j=1:MC_N
   ampo += eff_coup[j],"A",NP_Chain,"Adag",NP_Chain+j
   ampo += conj(eff_coup[j]),"Adag",NP_Chain,"A",NP_Chain+j
end


H = MPO(ampo,sysenv); 

#Set MPO bond dimensions
[dim(linkind(H,j)) for j in 1:length(H)-1];

In [8]:
psi=deepcopy(psi0)
timestep = 0.0001
tmax = 0.001

0.001

In [9]:
phi = tdvp(
   H, 
   -im*tmax, 
   psi0; 
time_step = -im*timestep,  
outputlevel = 1 , 
nsite = 1, 
normalize = false, 
ishermitian = false,
issymmetric = false,
solver_backend = "exponentiate"
);

Tempo exp avanti: 6.211611083
phi0 = ITensor ord=2
Dim 1: (dim=2|id=316|"Link,u")
Dim 2: (dim=8|id=598|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×8
 0.9999859067085977 - 0.0034499481577087965im                     0.0 + 0.0im                   -1.1513206283558038e-5 + 1.644170728360184e-7im                   0.0 + 0.0im                    5.4108571342151916e-11 - 1.4585686875228668e-12im                      0.0 + 0.0im                                       0.0 + 0.0im                    1.2655424130478817e-9 + 6.333212502268277e-8im
                0.0 + 0.0im                    -2.6491271346379713e-5 - 0.004035239907087281im                     0.0 + 0.0im                   5.28315834575867e-10 + 2.6821647673697032e-8im                     0.0 + 0.0im                     -3.2059057703183567e-15 - 9.763206269706325e-14im  -1.5696535442575857e-5 + 1.7447411568161122e-7im                    0.0 + 0.0im
Tempo exp avanti: 1.074505459
phi0 = ITensor ord=2
Dim 1: (dim=8|

Excessive output truncated after 527550 bytes.


Tempo exp avanti: 0.002898417
phi0 = ITensor ord=2
Dim 1: (dim=8|id=210|"Link,u")
Dim 2: (dim=8|id=864|"Link,u")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 8×8
      0.9999922167320052 + 0.0im   0.00028662996827706004 + 0.00048433755608268657im  -0.0004953964868450285 - 0.0010679051302723752im     -0.001183335115872843 + 0.0003555677189760627im     0.0009146059994477155 + 0.002789826416425193im     -0.0008106019038221702 + 0.0012372470110682222im   -0.00022494847324793147 + 0.0003241599550374207im    -0.000230532789636272 + 0.0011492903535734935im
  -8.194552851859969e-13 + 0.0im   -7.264671088213744e-10 + 1.5411131269377595e-9im   -7.718136957359963e-10 + 9.978464417686568e-10im   -2.6070742928623743e-10 - 5.231642976631274e-10im    3.0869290535263605e-10 + 3.4529200806531483e-10im  -2.2265135289456146e-10 - 4.1884929121637627e-10im   -2.086001961180766e-11 - 3.8941007123059836e-10im   2.840607036457311e-10 + 1.054129698190748e-10im
   4.442065370884383e-13 + 0.0im   -7.6590505

In [ ]:
dot(phi,psi0)

In [ ]:
using ITensors.HDF5
f = h5open("myfile.h5","w")
write(f,"psi",phi)
close(f)

In [ ]:
occ_closure = []
for j in 82:87
   orthogonalize!(phi,j); 
   phidag = dag(prime(phi[j],"Site"));
   m=real(scalar(phidag*op(sysenv,"N",j)*phi[j]))
   push!(occ_closure,m)
end

In [ ]:
occ_closure

In [ ]:
sqrt(dot(phi,phi))